In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("del2-vine").getOrCreate()

In [91]:
#tools for later
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
import pyspark.sql.functions as func

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://mod16-bkt.s3.us-east-2.amazonaws.com/data_tables/vine_table.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("vine_table.csv"), header=False, inferSchema=True)

# Show DataFrame
df.show()

+--------------+---+---+---+---+---+
|           _c0|_c1|_c2|_c3|_c4|_c5|
+--------------+---+---+---+---+---+
| RMDCHWD0Y5OZ9|  3|  0|  1|  N|  N|
| RZSL0BALIYUNU|  5|  0|  0|  N|  Y|
| RIZR67JKUDBI0|  3|  0|  1|  N|  Y|
|R27HL570VNL85F|  5|  0|  0|  N|  Y|
|R34EBU9QDWJ1GD|  5|  0|  0|  N|  Y|
|R1WCUI4Z1SIQEO|  5|  0|  0|  N|  N|
| RL5LNO26GAVJ1|  2|  3|  4|  N|  Y|
|R3GYQ5W8JHP8SB|  5|  0|  0|  N|  Y|
|R30SHYQXGG5EYC|  5|  0|  0|  N|  Y|
|R14YLXA56NP51I|  5|  1|  1|  N|  N|
|R1ZH0HSH38IOTZ|  5|  0|  0|  N|  Y|
|R3H53KLLC210XI|  4|  0|  0|  N|  Y|
|R3OOR877NGA8JK|  3|  0|  0|  N|  Y|
|R1BY7WKOZ3KMH0|  2|  0|  0|  N|  Y|
| RXP1TFSWE8EG9|  5|  0|  0|  N|  Y|
|R3J44DPP12OTLJ|  5|  0|  0|  N|  Y|
| RFOV69SK0T676|  5|  0|  0|  N|  Y|
|R2HUWDNW62FOL3|  1|  0|  0|  N|  Y|
|R1KSU30XZGR452|  5|  0|  0|  N|  Y|
|R2TZVLLTSHA07N|  5|  1|  1|  N|  Y|
+--------------+---+---+---+---+---+
only showing top 20 rows



In [8]:
#well i f#$%ed up the export while trying to save my work. so let's clean up that header
vine_df = df.withColumnRenamed('_c0','review_id').withColumnRenamed('_c1','star_rating').withColumnRenamed('_c2','helpful_votes').withColumnRenamed('_c3','total_votes').withColumnRenamed('_c4','vine').withColumnRenamed('_c5','verified_purchase')

vine_df.show(3)


+-------------+-----------+-------------+-----------+----+-----------------+
|    review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-------------+-----------+-------------+-----------+----+-----------------+
|RMDCHWD0Y5OZ9|          3|            0|          1|   N|                N|
|RZSL0BALIYUNU|          5|            0|          0|   N|                Y|
|RIZR67JKUDBI0|          3|            0|          1|   N|                Y|
+-------------+-----------+-------------+-----------+----+-----------------+
only showing top 3 rows



In [9]:
vine_filt_df = vine_df.filter(vine_df.total_votes >= 20)
vine_filt_df.show(3)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 3 rows



In [23]:
vine_calc = vine_filt_df.withColumn('vine_calc', (vine_filt_df.helpful_votes / vine_filt_df.total_votes) > 0.5)
vine_calc.show(3)
vine_calc.count()


+--------------+-----------+-------------+-----------+----+-----------------+---------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|vine_calc|
+--------------+-----------+-------------+-----------+----+-----------------+---------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|     true|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|     true|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|     true|
+--------------+-----------+-------------+-----------+----+-----------------+---------+
only showing top 3 rows



16520

In [24]:
helping_vines = vine_calc.filter(vine_calc.vine_calc == 'true')
helping_vines.show(3)
helping_vines.count()

+--------------+-----------+-------------+-----------+----+-----------------+---------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|vine_calc|
+--------------+-----------+-------------+-----------+----+-----------------+---------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|     true|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|     true|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|     true|
+--------------+-----------+-------------+-----------+----+-----------------+---------+
only showing top 3 rows



14496

In [25]:
#filter for paid
yesvine_df = helping_vines.filter(helping_vines.vine == 'Y')
yesvine_df.show(3)
yesvine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+---------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|vine_calc|
+--------------+-----------+-------------+-----------+----+-----------------+---------+
|R1R9RU7JW0MFR2|          4|           20|         23|   Y|                N|     true|
|R19EFYNN3W8Q07|          5|           26|         32|   Y|                N|     true|
|R34DJ1R8AEU0SG|          5|           29|         35|   Y|                N|     true|
+--------------+-----------+-------------+-----------+----+-----------------+---------+
only showing top 3 rows



60

In [26]:
#filter for unpaid
novine_df = helping_vines.filter(helping_vines.vine == 'N')
novine_df.show(3)
novine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+---------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|vine_calc|
+--------------+-----------+-------------+-----------+----+-----------------+---------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|     true|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|     true|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|     true|
+--------------+-----------+-------------+-----------+----+-----------------+---------+
only showing top 3 rows



14436

In [54]:
yvine_data1 = [(yesvine_df.count(), yesvine_df.filter(yesvine_df.star_rating == '5').count())]
yvine_schema = [StructField('yvine_count', IntegerType(), True), StructField('yvine_fivestars', IntegerType(), True)]
#yvine_schema = ['review_count','5star_reviews']
yvine_struct = StructType(fields=yvine_schema)

In [55]:
yvine_summ = spark.createDataFrame(yvine_data1, schema=yvine_struct, )
yvine_summ.show()
yvine_summ.dtypes

+-----------+---------------+
|yvine_count|yvine_fivestars|
+-----------+---------------+
|         60|             34|
+-----------+---------------+



[('yvine_count', 'int'), ('yvine_fivestars', 'int')]

In [59]:
yvine_pct = yvine_summ.withColumn('5star_pct', ((yvine_summ.yvine_fivestars / yvine_summ.yvine_count)*100))
yvine_pct.show()
yvine_pct.dtypes

+-----------+---------------+------------------+
|yvine_count|yvine_fivestars|         5star_pct|
+-----------+---------------+------------------+
|         60|             34|56.666666666666664|
+-----------+---------------+------------------+



[('yvine_count', 'int'), ('yvine_fivestars', 'int'), ('5star_pct', 'double')]

In [77]:
#after all that work I'm going to have to merge it all. I didn't understand the question. So let's make the novine_summ and just call that in the next cell
novine_data1 = [(novine_df.count(), novine_df.filter(novine_df.star_rating == '5').count())]
novine_schema = [StructField('novine_count', IntegerType(), True), StructField('novine_fivestars', IntegerType(), True)]
#yvine_schema = ['review_count','5star_reviews']
novine_struct = StructType(fields=novine_schema)

novine_summ = spark.createDataFrame(novine_data1, schema=novine_struct, )
novine_summ.show()
novine_summ.dtypes


+------------+----------------+
|novine_count|novine_fivestars|
+------------+----------------+
|       14436|            8208|
+------------+----------------+



[('novine_count', 'int'), ('novine_fivestars', 'int')]

In [94]:
tot_summ = [(helping_vines.count(), helping_vines.filter(helping_vines.star_rating == 5).count(),
             yesvine_df.count(),yesvine_df.filter(yesvine_df.star_rating == '5').count(), 
             (yesvine_df.filter(yesvine_df.star_rating == '5').count() / helping_vines.count())*100,
             novine_df.count(), novine_df.filter(novine_df.star_rating == '5').count(),
            (novine_df.filter(novine_df.star_rating == '5').count() / helping_vines.count())*100)]
tot_schema = [StructField('review_count', IntegerType(), True), StructField('fivestars', IntegerType(), True), StructField('vine_revs', IntegerType(), True), StructField('vine_fives', IntegerType(), True), StructField('vfive_pct', StringType(), True), StructField('novine_revs', IntegerType(), True), StructField('novine_fives', IntegerType(), True), StructField('novfive_pct', StringType(), True)]
tot_struct = StructType(fields=tot_schema)

In [95]:
summ_df = spark.createDataFrame(tot_summ, schema=tot_struct)
summ_df.show()

+------------+---------+---------+----------+------------------+-----------+------------+------------------+
|review_count|fivestars|vine_revs|vine_fives|         vfive_pct|novine_revs|novine_fives|       novfive_pct|
+------------+---------+---------+----------+------------------+-----------+------------+------------------+
|       14496|     8242|       60|        34|0.2345474613686534|      14436|        8208|56.622516556291394|
+------------+---------+---------+----------+------------------+-----------+------------+------------------+

